In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("complete")

/kaggle/input/cvalltogether/CV_brief.pdf
/kaggle/input/cvalltogether/drmperfect_cv-april-2020.pdf
/kaggle/input/cvalltogether/vita_external.pdf
/kaggle/input/cvalltogether/Klar CV.pdf
/kaggle/input/cvalltogether/blee_cv_2016.pdf
/kaggle/input/cvalltogether/Canales_Robert_CV.pdf
/kaggle/input/cvalltogether/MOORE-MONROY2015_0.pdf
/kaggle/input/cvalltogether/CURRICULUM-VITAE_DHG_012519.pdf
/kaggle/input/cvalltogether/RobertsonCV0818-2.pdf
/kaggle/input/cvalltogether/hameroff2016cv_0.pdf
/kaggle/input/cvalltogether/Alison-M-Meadow-cv.pdf
/kaggle/input/cvalltogether/agaspar_cv.pdf
/kaggle/input/cvalltogether/JO - 2171.pdf
/kaggle/input/cvalltogether/Hoit CV (4-11-16).pdf
/kaggle/input/cvalltogether/Liverman Selected CV May 2018.pdf
/kaggle/input/cvalltogether/LBarraza CV 2020.pdf
/kaggle/input/traindatacv/train_data.pkl
complete


In [2]:
!pip install PyPDF2
!pip install pdfplumber
print("complete")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.7/217.7 kB 480.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 395.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.0 MB/s eta 0:00:00
complete


In [3]:
#all import
import pdfplumber
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher

from datetime import datetime

import re
from nltk.corpus import stopwords

#to store data
allCVData=[]

print("complete")

complete


In [4]:
# to do : do partial match , i.e. word+etc

reference=[ 
            #node1
            "articles","article","books","book","chapters","chapter","citations","citation","editorials","editorial","journals","journal",
               "seminars","seminar","scholarly",
            #node2
            "awards","award","proposals","proposal","grants","grant","honors","honor","scholarships","scholarship","sponsored",
            #node3
            "appointments","appointment","experiences","experience","services","service","employments","employment","practices","practice",
                "professionals","professional",
            #node4
            "affiliations","affiliation","memberships","membership","committees","committee",
            #node5
            "contacts","contact","introductions","introduction",
            #node6
            "publications","publication","conferences","conference","presentations","presentation","newsletters","newsletter","reports","report",
            #node7
            "educations","education","certificates","certificate","certifications","certification",
            #node8
            "researches","research",
            #node9
            "teaching","outreaches","outreach"
          ]

#allCVData.append(("fileName","personName","sectionName","head","relation/lebel","tail"))

#print(allCVData)
#print(reference,len(reference))

print("complete")

complete


In [5]:
def dfPartByPart(textList):
    collectedText=[]
    collectedByDictionary=dict()
    collectedParts=dict()
    keyDictionary="introductions"
    temp=[]

    

    for smallPartText in textList:
        if smallPartText==None:
            continue
   
        for partOfSmallPartText in smallPartText.split():
            if len(partOfSmallPartText)<2:
                continue
            result = list(filter(lambda x: x==partOfSmallPartText.lower(), reference))
            
            if result:
                #print(result,keyDictionary,temp,smallPartText)
                #print(result,smallPartText)
                collectedText.append(temp)
                if keyDictionary in collectedByDictionary:
                    collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
                else:
                    collectedByDictionary[keyDictionary]=temp
                keyDictionary=result[0]
                #print(result,keyDictionary)
                temp=[]
                break
        
        temp.append(smallPartText)
    
    
    collectedText.append(temp)
    if keyDictionary in collectedByDictionary:
        collectedByDictionary[keyDictionary]=collectedByDictionary[keyDictionary]+temp
    else:
        collectedByDictionary[keyDictionary]=temp

    return collectedByDictionary
print("complete")

complete


In [6]:
#import re
#from nltk.corpus import stopwords
# load pre-trained model
#nlp = spacy.load('en_core_web_sm')
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))
# Education Degrees
EDUCATIONDEGREE = [
            'BE','B.E.', 'B.E', 'BS', 'B.S','B.S.','BSC','B.SC','B.SC.','C.A.','B.COM','BCOM',
            'M.COM', 'MCOM','M.COM.',
            'ME', 'M.E', 'M.E.', 'MS', 'M.S','M.S.','MSC','M.SC','M.SC.',
            'BTECH', 'B.TECH','B.TECH.', 'M.TECH','M.TECH.', 'MTECH',
            'PHD','PH.D', 'PH.D.','MBA','GRADUATE', 
            'POST-GRADUATE','MASTERS',
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
        ]
def extract_educationDegree(resume_text):
    nlp_text = nlp(resume_text)
    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    edu = []
    
    # Extract education degree
    for index, text in enumerate(nlp_text):
        text=text.replace(","," ")
        #print(text)
        for tex in text.split():
            # Replace all special symbols
            #tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            
            if tex.upper() in EDUCATIONDEGREE and tex not in STOPWORDS:
                
                if tex not in edu:
                    edu.append(tex)
                                
                
    return edu
print("complete")

complete


In [7]:
# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))
# Education Degrees
Dsignation = [
    'assistant professor','associate professor','assistant','affiliate','adjunct faculty',
    'associate research professor','associate research scientist','assistant specialist',
    'adjunct assistant research scientist',
    'instructor',
    'manager',
    'postdoctoral researcher','program manager','project manager','professor',
    'program evaluator','post-doctoral fellow','postdoctoral research fellowship',
    'research assistant','research technician',
    'senior research associate','staff scientist','seasonal position','specialist',
    'teaching assistant','teachers assistant'
        ]
def extractDsignation(resume_text):
    nlp_text = nlp(resume_text)
    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]
    edu = []
    
    # Extract education degree
    for index, text in enumerate(nlp_text):
        text=text.replace(","," ")
        if text.lower() in Dsignation and tex not in STOPWORDS:
                
            if text not in edu:
                edu.append(text)
        #print(text)
        for tex in text.split():
            # Replace all special symbols
            #tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            
            if tex.lower() in Dsignation and tex not in STOPWORDS:
                
                if tex not in edu:
                    edu.append(tex)
                                
                
    return edu
print("complete")

complete


In [8]:
def newGetDate(dataMaybeDate):
    match = re.search('\d{4}',dataMaybeDate)
    match001=re.search('\d{2}/\d{2}/\d{4}',dataMaybeDate)
    
    #if match001 is not None and int(match001)>1900 and int(match001)<2023:
    if match001:
        # Then it found a match!
        #print(type(match001))
        return match001.group(0)
    
    if match is not None :
        # Then it found a match!
        if int(match.group(0))>1900 and int(match.group(0))<2023:
            return match.group(0)
        return None
    return None
print("complete")

complete


In [9]:
def findInstitute(tika_text):
    sub_patterns = ['[A-Z][a-z]* University',
                    '[A-Z][a-z]* Educational Institute',
                '[A-Z][a-z]* College',
                'University of [A-Z][a-z]*',
                'The University of [A-Z][a-z]*',
                    'TheUniversityof[A-Z][a-z]*',
                'Ecole [A-Z][a-z]*',
                   '[A-Z][a-z]*University',
                    '[A-Z][a-z]*EducationalInstitute',
                '[A-Z][a-z]*College',
                'Universityof[A-Z][a-z]*',
                'Ecole[A-Z][a-z]*',
                    'The [A-Z][a-z]* Academy of [A-Z][a-z]*',
                    'the [A-Z][a-z]* academy of [A-Z][a-z]*'
                   ]
    pattern = '({})'.format('|'.join(sub_patterns))
    matches = re.findall(pattern, tika_text)

    return matches

print("complete")

complete


In [10]:
def removePrefix(dataSent):
    resDataIndex=0
    for i in dataSent:
        #if i.isalpha():
        if i.isalnum():
            #print(i,resDataIndex)
            break
        resDataIndex+=1
    dataSent=dataSent[resDataIndex:]
    return dataSent
print("complete")

complete


In [11]:
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match = None)
    
    matches = matcher(nlp_text)
    res=[]
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        temp=span.text
        if len(temp)>0:
            lastoption=temp
        if ("vitae" in temp.lower() or "sciences" in temp.lower() 
            or "engineering" in temp.lower() or "biographical" in temp.lower() 
            or "no" in temp.lower() or "title" in temp.lower() ):
            continue
        res.append(temp)
    
    return res
print("complete")

complete


In [12]:
#E-MAIL
#import re
def get_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)


print("complete")

complete


In [13]:
def handleEducations(listOfEducations):
    eduData=[]
    pieceEduData=dict()
    rest=""
    for eduPart in listOfEducations:
        #print(type(eduPart))
        
        res=extract_educationDegree(eduPart)
        if res:
            if len(pieceEduData)!=0:
                if len(rest)>1:
                    pieceEduData["total details"]=rest
                    rest=""
                eduData.append(pieceEduData)
                pieceEduData=dict()
            pieceEduData["degree"]=res[0]
            index001=eduPart.find(res[0])
            eduPart=eduPart[:index001]+eduPart[index001+len(res[0]):]
            #print(res,index001,eduPart)
            #print(res,eduPart)
        #datePart=getDate(eduPart)
        newDatePart=newGetDate(eduPart)
        
        #if datePart:
        #    print("date:",datePart)
        if newDatePart:
            pieceEduData["date"]=newDatePart
            index002=eduPart.find(newDatePart)
            eduPart=eduPart[:index002]+eduPart[index002+len(newDatePart):]
            #print(index002,"date:",newDatePart,eduPart)
        
        #test001=eduPart.split(":")
        #for itest in test001:
        if "advisor" in eduPart.lower():
            
            index003=eduPart.lower().find("advisor")
            advis001=eduPart[index003+len("advisor"):]
            advis001=removePrefix(advis001)
            pieceEduData["advisor"]=advis001
            eduPart=eduPart[:index003]
            #print("advisor",advis001,eduPart)
        org=findInstitute(eduPart)
        if org:
            pieceEduData["organization"]=org[0]
            index004=eduPart.find(org[0])
            eduPart=eduPart[:index004]+eduPart[index004+len(org[0]):]
            #print(org,eduPart)
        #print(eduPart)
        eduPart=removePrefix(eduPart)
        
        designation=extractDsignation(eduPart)
        if designation:
            pieceEduData["designation"]=designation[0]
            index004=eduPart.find(designation[0])
            eduPart=eduPart[:index004]+eduPart[index004+len(designation[0]):]
        
        if len(eduPart)>1:
            rest+=eduPart
        #print(eduPart)
        
        
    if len(pieceEduData)!=0:
        if len(rest)>1:
            pieceEduData["total details"]=rest
                    #rest=""
        eduData.append(pieceEduData)
    return eduData
print("complete")

complete


In [14]:
def handleContacts(sentDataContacts):
    
    contactDetails=dict()
    
    #otherDetails=[]
    
    #print(sentDataContacts)
    textContacts001=",".join(sentDataContacts)
    nameContacts001=extract_name(textContacts001)
    if nameContacts001:
        contactDetails["name"]=nameContacts001[0]
    else:
        contactDetails["name"]=sentDataContacts[0]
    
    #flagContact=False
    othersRests=[]
    #print(nameContacts001[0],nameContacts001)
    for contacts00 in sentDataContacts:
        #dateFound001=newGetDate(contacts00)
        #print(dateFound001,contacts00)
        
        
        
        e_mailContacts=get_email_addresses(contacts00)
        if e_mailContacts:
            #print(e_mailContacts,contacts00)
            #contactIndex001=contacts00.lower().find(e_mailContacts)
            contactIndex001=contacts00.find(e_mailContacts[0])
            
            contacts00=contacts00[:contactIndex001]+contacts00[contactIndex001+len(e_mailContacts[0]):]
            contacts00=removePrefix(contacts00)
            
            
            contactDetails["e-mails"]=e_mailContacts[0]
            #print(e_mailContacts,contacts00)
        othersRests.append(contacts00)
    contactDetails["other_details"]=" ".join(othersRests)
    return contactDetails
print("complete")

complete


In [15]:
def handleRestPart(experiencePart):
    allExperience=[]
    detailsExperience=""
    tempExperience=dict()
    flag=False # need for different in the 1st
    for partExperience in experiencePart:
        dateFound=newGetDate(partExperience)
        if dateFound:
            
            
            if not flag:
                flag=True
                tempExperience['year']=dateFound
                partExperienceIndex001=partExperience.find(dateFound)
            
                partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(dateFound):]
                partExperience=removePrefix(partExperience)
                detailsExperience+=" "+partExperience
                
                organizationExperience=findInstitute(partExperience)
        
                if organizationExperience:
                
                    partExperienceIndex001=partExperience.find(organizationExperience[0])
            
                    partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(organizationExperience[0]):]
                    partExperience=removePrefix(partExperience)
                
                    tempExperience['organization']=organizationExperience[0]
                
                dsignationExperience=extractDsignation(partExperience)
                if dsignationExperience:
                
                    partExperienceIndex001=partExperience.find(dsignationExperience[0])
            
                    partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(dsignationExperience[0]):]
                    partExperience=removePrefix(partExperience)
                
                    tempExperience['dsignation']=dsignationExperience[0]
                
                continue
            else:
                tempExperience['details']=detailsExperience
                detailsExperience=""
                allExperience.append(tempExperience)
                
                tempExperience=dict()
                tempExperience['year']=dateFound
                
                partExperienceIndex001=partExperience.find(dateFound)
            
                partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(dateFound):]
                partExperience=removePrefix(partExperience)
                
                organizationPublications=findInstitute(partExperience)
        
                if organizationPublications:
                
                    partExperienceIndex001=partExperience.find(organizationPublications[0])
            
                    partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(organizationPublications[0]):]
                    partExperience=removePrefix(partExperience)
                
                    tempExperience['organization']=organizationPublications[0]
                dsignationExperience=extractDsignation(partExperience)
                if dsignationExperience:
                
                    partExperienceIndex001=partExperience.find(dsignationExperience[0])
            
                    partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(dsignationExperience[0]):]
                    partExperience=removePrefix(partExperience)
                
                    tempExperience['dsignation']=dsignationExperience[0]
                
                detailsExperience+=" "+partExperience
        else:
            partExperience=removePrefix(partExperience)
            
            organizationPublications=findInstitute(partExperience)
        
            if organizationPublications:
                
                partExperienceIndex001=partExperience.find(organizationPublications[0])
            
                partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(organizationPublications[0]):]
                partExperience=removePrefix(partExperience)
                
                tempExperience['organization']=organizationPublications[0]
            
            
            dsignationExperience=extractDsignation(partExperience)
            if dsignationExperience:
                
                partExperienceIndex001=partExperience.find(dsignationExperience[0])
            
                partExperience=partExperience[:partExperienceIndex001]+partExperience[partExperienceIndex001+len(dsignationExperience[0]):]
                partExperience=removePrefix(partExperience)
                
                tempExperience['dsignation']=dsignationExperience[0]
            
            detailsExperience+=" "+partExperience
        
        
        
        
        
        # extracting name gives a lot of false result, 
        #as it contains name of things other than person name
        
        #namepartExperience=extract_name(partExperience)
        #if namepartExperience:
            #print(namepartExperience,type(namepartExperience))
    
        
        
    if tempExperience:
        tempExperience['details']=detailsExperience
        allExperience.append(tempExperience)
            
    return allExperience
print("complete")

complete


In [16]:
#import pdfplumber
#import pandas as pd
#import os
# Importing required modules
import PyPDF2

def extract_pdf(pdf_path):
    linesOfFile = []
    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            for line in single_page_text.split('\n'):
                linesOfFile.append(line)
                #print(linesOfFile)
    return linesOfFile


folder_with_pdfs = '../input/cvalltogether'
linesOfFiles = []

#specially for testing ner
dataForTest=dict()

listOfPdfFiles=[]

for pdf_file in os.listdir(folder_with_pdfs):
    if pdf_file.endswith('.pdf'):
        
        #print(pdf_file)
        listOfPdfFiles.append(pdf_file)
        pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        
        pdfFileObj = open(pdf_file_path,'rb')
        
        #print(pdfFileObj)
        
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# Getting number of pages in pdf file
        pages = pdfReader.numPages

        totalText=[]

# Loop for reading all the Pages
        for i in range(pages):

        # Creating a page object
            pageObj = pdfReader.getPage(i)

        # Printing Page Number
        #print("Page No: ",i)

        # Extracting text from page
        # And splitting it into chunks of lines
            text = pageObj.extractText().split("\n")
        
            totalText+=text
        
        
# closing the pdf file object
        pdfFileObj.close()
        #print(totalText)
        
        #sectionDictionary=dfPartByPart(totalText)
        #newSectionDictionary=reDistribute(sectionDictionary)
        
        
        newSectionDictionary=dfPartByPart(totalText)
        
        #specially for testing ner
        dataForTest=newSectionDictionary
        
        tempData00=[]
        
        contactPart001,educationsPart001=dict(),dict()
        
        personName00=""
        personNameLast=""
        if 'contacts' in newSectionDictionary:
            text00=newSectionDictionary['contacts']
            personName00=extract_name(",".join(text00))
            personNameLast=personName00[0]
        
        
        if len(personName00)==0:
            if 'introductions' in newSectionDictionary:
                text00=newSectionDictionary['introductions']
                personName00=extract_name(",".join(text00))
            
                
            if len(personName00)==0:
                personNameLast="no name"
            else:
             
            
                personNameLast=personName00[0]
        else:
             
            
            personNameLast=personName00[0]
        #print(i,personNameLast)

        
        
        for partSections in newSectionDictionary:
            tempData00=[personNameLast]
            if (partSections=='contacts' or partSections=='introductions') and newSectionDictionary[partSections]:
                tempData00.append(partSections+" details")
                tempData00.append(partSections)
                contactPart001=handleContacts(newSectionDictionary[partSections])
                for parts in contactPart001:
                    tempData00.append(parts)
                    tempData00.append(contactPart001[parts])
                
                #print(tempData00)
                    if len(tempData00)>1:
                        allCVData.append(tempData00)
                
                    tempData00.pop()
                    tempData00.pop()
            
                tempData00.pop()
                tempData00.pop()
            
            #print(contactPart001) 
            #done for now
            elif (partSections=='educations' or partSections=='certificates' or partSections=='certifications') and newSectionDictionary[partSections]:
                tempData00.append("qualifications details")
                tempData00.append(partSections)
                educationsPart001=handleEducations(newSectionDictionary[partSections])
                for parts001 in educationsPart001:
                    tempData00.append("degree earned")
                    if 'degree' in parts001:
                        tempData00.append(parts001['degree'])
                    else:
                        tempData00.append("Degree")
                
                    for smallparts002 in parts001:
                        tempData00.append(smallparts002)
                        tempData00.append(parts001[smallparts002])
                    
                    #print(tempData00)
                        if len(tempData00)>1:
                            allCVData.append(tempData00.copy())
                    
                        tempData00.pop()
                        tempData00.pop()
                    tempData00.pop()
                    tempData00.pop()
                tempData00.pop()
                tempData00.pop()
            else:
                tempEdge=partSections +" experience"
        
                tempData00.append(tempEdge)
                tempData00.append(partSections)
                restPart001=handleRestPart(newSectionDictionary[partSections])
        
                count=1
                for parts001 in restPart001:
                    newTempEdge=partSections +" details"
                    tempData00.append(newTempEdge)
                    tempData00.append(partSections+" no."+str(count))
            
            
                
                    for smallparts002 in parts001:
                        tempData00.append(smallparts002)
                        tempData00.append(parts001[smallparts002])
                    
                    #print(tempData00)
                        if len(tempData00)>1:
                            allCVData.append(tempData00.copy())
                    
                        tempData00.pop()
                        tempData00.pop()
                    tempData00.pop()
                    tempData00.pop()
                    count+=1
                tempData00.pop()
                tempData00.pop()
#print("complete")
        #break
        
        
        
        
        break
        
#df = pd.DataFrame(linesOfFiles)
#df.to_csv('test.csv')
print("complete")

complete


In [17]:
totalAllCVData=[]
for check in allCVData:
    if len(check)>1:
        totalAllCVData.append(check)
print(totalAllCVData[:10])

print("complete")

[['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.1', 'year', '1995'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.1', 'organization', 'York University'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.1', 'details', ' Education Ph.D. Courant Institute, New York University, New York, NY.'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.2', 'year', '1990'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.2', 'organization', 'Stanford University'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.2', 'details', ' B.S. , Stanford, CA.'], ['Sunder Sethuraman', 'education experience', 'education', 'education details', 'education no.3', 'year', '1986'], ['Sunder Sethuraman', 'education experience', 'education', 'edu

In [18]:
import csv

#write.csv(allCVData, "mycsv.csv")

#header = ['head','relation','tail']

with open('CVpdftotalAllCVData001.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header , already has a header
    #writer.writerow(header)

    # write multiple rows
    writer.writerows(totalAllCVData)

print("complete")

complete


In [19]:
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load('en')
def nerCV(personName,sectionName,sentDataFoeTest):
    allDataner=[]
    metadata001=[personName,sectionName+" with",sectionName]
    countNo=0
    for train_dataText in sentDataFoeTest :
        
    
    #train_dataText=" ".join(sentDataFoeTest)
    #doc = loaded_nlp_model(train_dataText)
    
        doc = nlp(train_dataText)
        #print(doc)
        
        dldata=dict()
    #print(doc)
        #print("*************")
        for ent in doc.ents:
        #print("as")
            if ent.label_.upper() in dldata:
                dldata[ent.label_.upper()].append(ent.text)
            else:
                dldata[ent.label_.upper()]=[ent.text]
            #print(ent.label_.upper())
            #print(f'{ent.label_.upper():{30}}- {ent.text}')
    #break
        if dldata:
            countNo+=1
            #print(dldata)
            #$allDataner.append(dldata)
            for dl in dldata:
                metadata=metadata001.copy()
                if dl=='DATE':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " in "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        #print(metadata)
                        metadata.pop()
                        metadata.pop()
                elif dl=='ORG':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " with "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                elif dl=='PERSON':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " associated with "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                elif dl=='GPE':
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " from "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
                else:
                    for d in dldata[dl]:
                        if len(d)<3:
                            continue
                        metadata.append(sectionName+ " in "+str(dl)+" "+str(countNo)+" ")
                        metadata.append(d)
                        allDataner.append(metadata.copy())
                        metadata.pop()
                        metadata.pop()
            #
            metadata.append("Original text "+str(countNo)+" ")
            metadata.append(train_dataText)
            allDataner.append(metadata.copy())
            metadata.pop()
            metadata.pop()
    #print(allDataner)
    return allDataner
print("complete")

complete


In [20]:
folder_with_pdfs = '../input/cvalltogether'
linesOfFiles = []

#specially for testing ner
dataForTest=dict()

listOfPdfFiles=[]

alnerCVData=[]

for pdf_file in os.listdir(folder_with_pdfs):
    if pdf_file.endswith('.pdf'):
        
        #print(pdf_file)
        listOfPdfFiles.append(pdf_file)
        pdf_file_path = os.path.join(folder_with_pdfs, pdf_file)
        
        pdfFileObj = open(pdf_file_path,'rb')
        
        #print(pdfFileObj)
        
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# Getting number of pages in pdf file
        pages = pdfReader.numPages

        totalText=[]

# Loop for reading all the Pages
        for i in range(pages):

        # Creating a page object
            pageObj = pdfReader.getPage(i)

        # Printing Page Number
        #print("Page No: ",i)

        # Extracting text from page
        # And splitting it into chunks of lines
            text = pageObj.extractText().split("\n")
        
            totalText+=text
        
        
# closing the pdf file object
        pdfFileObj.close()
        #print(totalText)
        
        #sectionDictionary=dfPartByPart(totalText)
        #newSectionDictionary=reDistribute(sectionDictionary)
        
        
        newSectionDictionary=dfPartByPart(totalText)
        
        personName00=""
        personNameLast=""
        if 'contacts' in newSectionDictionary:
            text00=newSectionDictionary['contacts']
            personName00=extract_name(",".join(text00))
            personNameLast=personName00[0]
        
        
        if len(personName00)==0:
            if 'introductions' in newSectionDictionary:
                text00=newSectionDictionary['introductions']
                personName00=extract_name(",".join(text00))
            
                
            if len(personName00)==0:
                personNameLast="no name"
            else:
             
            
                personNameLast=personName00[0]
        else:
             
            
            personNameLast=personName00[0]
        
        #specially for testing ner
        dataForTest=newSectionDictionary
        for sectionDataSentTest in dataForTest:
            if len(dataForTest[sectionDataSentTest])>1:
                nerret=nerCV(personNameLast,sectionDataSentTest,dataForTest[sectionDataSentTest][1:])
                #print(nerret)
                #alnerCVData.extend()
                for i in nerret:
                    if i not in alnerCVData:
                        alnerCVData.append(i)
                
#dataForTest
        #break
#alnerCVData=list(set(alnerCVData))
#print(alnerCVData)
print("complete")

complete


In [21]:
print(alnerCVData[:10])
print("complete")

[['Sunder Sethuraman', 'introductions with', 'introductions', 'introductions with ORG 1 ', 'WWW'], ['Sunder Sethuraman', 'introductions with', 'introductions', 'Original text 1 ', 'WWW: http://www.math.arizona.edu/~ sethuram/'], ['Sunder Sethuraman', 'education with', 'education', 'education in DATE 1 ', '1995'], ['Sunder Sethuraman', 'education with', 'education', 'education with ORG 1 ', 'Ph.D. Courant Institute'], ['Sunder Sethuraman', 'education with', 'education', 'education with ORG 1 ', 'New York University'], ['Sunder Sethuraman', 'education with', 'education', 'education from GPE 1 ', 'New York'], ['Sunder Sethuraman', 'education with', 'education', 'Original text 1 ', '1995 Ph.D. Courant Institute, New York University, New York, NY.'], ['Sunder Sethuraman', 'education with', 'education', 'education in DATE 2 ', '1990'], ['Sunder Sethuraman', 'education with', 'education', 'education with ORG 2 ', 'B.S. Stanford University'], ['Sunder Sethuraman', 'education with', 'education'

In [22]:
for sectionDataSentTest in dataForTest:
    if len(dataForTest[sectionDataSentTest])>1:
        nerret=nerCV(personNameLast,sectionDataSentTest,dataForTest[sectionDataSentTest][1:])
        
        #print(nerret)

In [23]:
with open('nerCVpdftotalAllCVData001.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header , already has a header
    #writer.writerow(header)

    # write multiple rows
    writer.writerows(alnerCVData)


print("complete")

complete
